In [966]:
import pandas
import json
import numpy
from pandas import ExcelWriter
import os

In [967]:
path = os.path.abspath(os.getcwd())

In [968]:
def clean_column_names(cols):
    cols = cols.str.replace(r'[^\x00-\x7F]+', " ", regex=True)\
                  .str.replace(" +", " ", regex=True)\
                  .str.strip()\
                  .str.replace(" ", "_")\
                  .str.upper()
    return cols

In [975]:
def set_types_fillna(df1, cols):
#     can handle str, float, bool
#     cant handle datetime
    for col, spec in cols.items():
        if spec.get("data_type") == 'str':
            df1[col] = df1[col].fillna(spec.get("default")).astype(spec.get("data_type"))
        else:
            df1[col] = df1[col].astype(spec.get("data_type")).fillna(spec.get("default"))
            
    return df1

In [970]:
def save_xls(dict_df, path):
    """
    Save a dictionary of dataframes to an excel file, with each dataframe as a seperate page
    """
    writer = ExcelWriter(path)
    for key in dict_df:
        dict_df[key].to_excel(writer, key, index=False)

    writer.save()

In [971]:
def generate_diff(df1, df2):
    df_all = pandas.concat([df1, df2], axis='columns', keys=['First', 'Second'])
    df_final = df_all.swaplevel(axis='columns')[df1.columns[:]]

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('First', axis='columns', level=-1)
        return pandas.DataFrame(numpy.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    df_final = df_final.style.apply(highlight_diff, axis=None)
    return df_final

In [972]:
with open('config.json') as f:
    config_data = json.load(f)

In [973]:
for conf in config_data:
    filename = conf.get("filename")
    sheets = conf.get("sheet_names")
    xls = pandas.ExcelFile(filename)
    df_dict = {}
    df_diff_dic = {}
    for sheet_name, conf_cols in sheets.items():
        df_out, df_diff = pipeline(filename, sheet, xls, conf_cols)
        df_dict[sheet] = df_out
#         df_diff_dic[sheet] = df_diff
    save_xls(df_dict, os.path.join(path, filename.split('.')[0] + "_output.xlsx"))
#     some issue with writing the styler object, skipping diff output for now
#     save_xls(df_diff_dic, os.path.join(path, filename.split('.')[0] + "_diff.xlsx"))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [974]:
def pipeline(filename, sheetname, xls, cols_conf):
    columns = list(cols_conf.get("columns").keys())
    
    df0  = pandas.read_excel(xls, sheet_name=sheet_name)
    df1 = df0
    df1.columns = clean_column_names(df1.columns)
    df1 = df1[[i for i in columns]]
    df1 = set_types_fillna(df1, cols_conf.get("columns"))
    df_diff = generate_diff(df0, df1)
    return df1, df_diff

In [982]:
df0['ID']
# .fillna(0).astype(float)

0      1562
1      1582
2      2587
3      3549
4      2468
       ... 
696    6525
697    3265
698       -
699    6125
700    1562
Name: ID, Length: 701, dtype: object

In [983]:
pandas.to_numeric(df0["ID"], downcast="float")

ValueError: Unable to parse string "       " at position 224